In [1]:
import sys
sys.version

'3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]'

In [3]:
!git clone https://github.com/airockchip/rknn-llm --depth=1

Cloning into 'rknn-llm'...


Updating files: 100% (548/548), done.



In [2]:
#!pip install /content/rknn-llm/rkllm-toolkit/rkllm_toolkit-1.2.0-cp311-cp311-linux_x86_64.whl
!pip install rknn-llm/rkllm-toolkit/rkllm_toolkit-1.2.0-cp310-cp310-linux_x86_64.whl

Processing ./rknn-llm/rkllm-toolkit/rkllm_toolkit-1.2.0-cp310-cp310-linux_x86_64.whl
rkllm-toolkit is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [ ]:
from rkllm.api import RKLLM

In [ ]:
llm = RKLLM()

In [ ]:
import numpy as np
import torch

from huggingface_hub import snapshot_download, hf_hub_download

In [ ]:
token = ""
try:
    from google.colab import userdata
    token = userdata.get("HF_TOKEN")
except:
    token = input("Please enter your Hugging Face token: ")  

## Model Build from HuggingFace

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
if False:
  modelPath = 'OrpheusMain'  # "/content/OrpheusMain"
  snapshot_download(repo_id="Prince-1/orpheus_3b_0.1_ft_16bit", local_dir=modelPath) #, token= token)
  
  ret = llm.load_huggingface(model=modelPath, model_lora = None,device=device)
  if ret != 0:
    print('Load model failed!')
    exit(ret)


INFO: rkllm-toolkit version: 1.2.0
ERROR: Only quantized models exported by auto-gptq are supported.


Load model failed!


## Model Build from GGUF

In [ ]:



if True:
    file_path = hf_hub_download(repo_id="Prince-1/orpheus_3b_0.1_ft_16bit",filename="unsloth.F16.gguf") #, token = userdata.get("HfKey"))

    ret = llm.load_gguf(model='GGUF/unsloth.F16.gguf')
    if ret != 0:
        print('Load model failed!')
        exit(ret)

## Build model

In [ ]:

dataset = None
qparams = None
target_platform = "RK3588"
optimization_level = 1
quantized_dtype = "w8a8" #"w4a16_g32" #w4a16_g64 or w4a16_g128
quantized_algorithm = "normal"
num_npu_core = 3

ret = llm.build(
    do_quantization=False,optimization_level=optimization_level,
    quantized_dtype=quantized_dtype,quantized_algorithm=quantized_algorithm,
    target_platform=target_platform, num_npu_core=num_npu_core,
    extra_qparams=qparams, dataset=dataset)
if ret != 0:
  print('Build model failed!')
  exit(ret)


## Export rkllm model

In [ ]:

ret =llm.export_rkllm(f"orpheus-3b-0.1-ft_{quantized_dtype}_{target_platform}.rkllm")
if ret != 0:
  print('Export model failed!')
  exit(ret)